In [1]:
pip install psycopg2-binary

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 1.9 MB/s eta 0:00:01
   --- ------------------------------------ 0.1/1.2 MB 1.6 MB/s eta 0:00:01
   ---- ----------------------------------- 0.1/1.2 MB 1.2 MB/s eta 0:00:01
   ------ --------------------------------- 0.2/1.2 MB 1.2 MB/s eta 0:00:01
   --------- ------------------------------ 0.3/1.2 MB 1.4 MB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.2 MB 1.4 MB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.2 MB 1.4 MB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.2 MB 1.4 MB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.2 MB 933.2 kB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.2 MB 933.2 kB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.2 MB 933.2 kB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.2 MB 933.2 kB/s eta 0:00:01
   --------

In [2]:
import boto3
import psycopg2

In [ ]:
import json

with open('creds.json') as config_file:
    config = json.load(config_file)
    access_key = config['aws_access_key_id']
    secret_key = config['aws_secret_access_key']
    api_access_key = config['api_key']


In [ ]:
bucket_name = "capstone-etl-bucket"
csv_file_path = "weather_data.csv"
s3_key = "weather_today"
aws_s3_region = "us-east-1"

In [ ]:
# Define AWS and Redshift connection details
AWS_ACCESS_KEY = access_key
AWS_SECRET_KEY = secret_key
REDSHIFT_HOST = 'capstone-project.cedwcaggje2k.us-east-1.redshift.amazonaws.com:5439/mydb'
REDSHIFT_PORT = '5439'
REDSHIFT_DBNAME = 'mydb'
REDSHIFT_USER = 'dokes'
REDSHIFT_PASSWORD = 'Password1002'
IAM_ROLE = 'dokes'

In [ ]:
# S3 bucket details
S3_BUCKET = bucket_name
S3_KEY = s3_key
REGION =   aws_s3_region

# Redshift table details
TABLE_NAME = 'your_redshift_table_name'

# Establish connection to Redshift
try:
    conn = psycopg2.connect(
        dbname=REDSHIFT_DBNAME,
        user=REDSHIFT_USER,
        password=REDSHIFT_PASSWORD,
        host=REDSHIFT_HOST,
        port=REDSHIFT_PORT
    )
    print("Connected to Redshift successfully!")
except Exception as e:
    print(f"Unable to connect to Redshift: {e}")

# Create a cursor object
cur = conn.cursor()

# Construct the COPY command
copy_command = f"""
COPY {TABLE_NAME}
FROM 's3://{S3_BUCKET}/{S3_KEY}'
IAM_ROLE '{IAM_ROLE}'
REGION '{REGION}'
FORMAT AS CSV
IGNOREHEADER 1
"""

# Execute the COPY command
try:
    cur.execute(copy_command)
    conn.commit()
    print("Data copied to Redshift successfully!")
except Exception as e:
    conn.rollback()
    print(f"Error executing COPY command: {e}")

# Close the cursor and connection
cur.close()
conn.close()